# rapyuta.inout
- [write_fits](#write_fits), [read_fits](#read_fits),
- [arr2tab](#arr2tab), [tab2arr](#tab2arr), [get_cd](#get_cd), [get_pc](#get_pc), [patch_wcs_3D](#patch_wcs_3D)
- [write_hdf5](#write_hdf5), [read_hdf5](#read_hdf5)
- [write_ascii](#write_ascii), [read_ascii](#read_ascii)
- [write_csv](#write_csv), [read_csv](#read_csv)
- [fclean](#fclean)

In [25]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys
from pathlib import Path
import numpy as np
from pprint import pprint
from astropy.io import fits
from astropy.wcs import WCS

## Local
import rapyuta.utbox as UT
from rapyuta.inout import (
    fclean, fitsext, h5ext, ascext, csvext, savext,
    read_fits, write_fits,
    arr2tab, tab2arr, get_cd, get_pc, patch_wcs_3D,
    read_hdf5, write_hdf5,
    read_ascii, write_ascii, read_csv, write_csv,
)

## fold inputs
# UT.codefold(True, 'onclick')

/Users/dhu/Github/RAPYUTA/man/..


In [2]:
## Set dir
cfd = Path('').parent.absolute() # current file dir
datdir = cfd/'lib'
# outdir = cfd/'out'
outdir = cfd/'out/test_inout'
outdir.mkdir(parents=True, exist_ok=True)

datdir = str(datdir)
outdir = str(outdir)

file_sip_free = datdir+'/M82_04_LL3'+fitsext # no SIP kw
file_sip = datdir+'/M82_04_SL3'+fitsext # with SIP kw

filout = outdir+'/test_inout'

## Prepare data

In [3]:

cd = np.array(((0.00035731, 0.00036933), (0.00036933, -0.00035731)))

## get_cd

In [4]:
## Prepare data
hdr = fits.open(file_sip_free)[0].header
pc = np.array([(hdr['PC1_1'], hdr['PC2_1']), (hdr['PC1_2'], hdr['PC2_2'])])
cdelt = np.array((hdr['CDELT1'], hdr['CDELT2']))

## via pc+cdelt
get_cd_0 = get_cd(pc, cdelt)
cd = get_cd_0.cd
print('pc\n', pc)
print('cdelt\n', cdelt)
print('cd (via pc+cdelt)\n', get_cd_0.cd)
print('\n')

## via header
get_cd_1 = get_cd(header=hdr)
print('pc (via header)\n', get_cd_1.pc)
print('cdelt (via header)\n', get_cd_1.cdelt)
print('cd (via header)\n', get_cd_1.cd)
print('\n')

## via wcs
w = WCS(file_sip_free, naxis=2)
get_cd_2 = get_cd(wcs=w)
print('pc (via wcs)\n', get_cd_2.pc)
print('cdelt (via wcs)\n', get_cd_2.cdelt)
print('cd (via wcs)\n', get_cd_2.cd)

pc
 [[-0.74958509 -0.661908  ]
 [ 0.661908   -0.74958509]]
cdelt
 [-0.00141111  0.00141111]
cd (via pc+cdelt)
 [[ 0.00105775  0.00093403]
 [ 0.00093403 -0.00105775]]


pc (via header)
 [[-0.74958509  0.661908  ]
 [-0.661908   -0.74958509]]
cdelt (via header)
 [-0.00141111  0.00141111]
cd (via header)
 [[ 0.00105775 -0.00093403]
 [-0.00093403 -0.00105775]]


pc (via wcs)
 [[-0.74958509  0.661908  ]
 [-0.661908   -0.74958509]]
cdelt (via wcs)
 [-0.00141111  0.00141111]
cd (via wcs)
 [[ 0.00105775 -0.00093403]
 [-0.00093403 -0.00105775]]


## get_pc

In [5]:
print('pc (via cd)\n', get_pc(cd).pc)

pc (via cd)
 [[-0.74958509 -0.661908  ]
 [ 0.661908   -0.74958509]]


## read_fits

In [6]:
get_fdata = read_fits(datdir+'/M82_04_SL3')
# print('header \n', get_fdata.header)
# print('header of TAB \n', get_fdata.header_w)
print('data (shape) \n', get_fdata.data.shape)
print('wave (shape) \n', get_fdata.wave.shape)
print('uncertainty (shape) \n', get_fdata.unc.shape)
print('\n')

get_jdata = read_fits(datdir+'/VV114_ch1_long_s3d.fits',
                      wmod=1, instr=None, instr_auto=True, ext='')
# print('header \n', get_jdata.header)
# print('header of TAB \n', get_jdata.header_w)
print('JWST data (shape) \n', get_jdata.data.shape)
print('JWST uncertainty (shape) \n', get_jdata.unc.shape)

data (shape) 
 (22, 30, 81)
wave (shape) 
 (22,)
uncertainty (shape) 
 (22, 30, 81)


JWST data (shape) 
 (1120, 41, 45)
JWST uncertainty (shape) 
 (1120, 41, 45)


/Users/dhu/Github/RAPYUTA/rapyuta/inout.py:198: UserWarning: 2D WCS extracted from 3D data! 
  warnings.warn('2D WCS extracted from 3D data! ')


## tab2arr

In [7]:
jwave = tab2arr(get_jdata.wave)
print('JWST wave (shape) \n', jwave.shape)

JWST wave (shape) 
 (1120,)


## arr2tab

In [8]:
wave = arr2tab(get_fdata.wave, header=get_fdata.header_w)
wave

FITS_rec([([[7.3675313], [7.429502 ], [7.4914727], [7.553444 ], [7.6154146], [7.6773853], [7.7393565], [7.801327 ], [7.863298 ], [7.925269 ], [7.98724  ], [8.049211 ], [8.111181 ], [8.173153 ], [8.235124 ], [8.297094 ], [8.359065 ], [8.421036 ], [8.4830065], [8.544978 ], [8.606949 ], [8.66892  ]],)],
         dtype=(numpy.record, [('WAVE-TAB', '<f4', (22, 1))]))

## write_fits

In [9]:
write_fits(outdir+'/sip_cube_rewriten',
           get_fdata.header, get_fdata.data, wave, header_w=get_fdata.header_w)
write_fits(outdir+'/jcube_irs_data_format_copy',
           get_jdata.header, get_jdata.data, jwave, wmod=1)

## patch_wcs_3D

In [10]:
print('Tested with astropy v5.1')
print('********************')
print(WCS(file_sip_free))
# print(WCS(file_sip)) # With SIP kw
print('\n')

#ValueError:
#FITS WCS distortion paper lookup tables and SIP distortions only work in 2 dimensions.
#However, WCSLIB has detected 3 dimensions in the core WCS keywords.
#To use core WCS in conjunction with FITS WCS distortion paper lookup tables or SIP distortion, 
#you must select or reduce these to 2 dimensions using the naxis kwarg.

Tested with astropy v5.1
********************
WCS Keywords

Number of WCS axes: 3
CTYPE : 'RA---TAN'  'DEC--TAN'  'WAVE-TAB'  
CRVAL : 149.024706598  69.6615857438  1.0  
CRPIX : 37.5  6.5  1.0  
PC1_1 PC1_2 PC1_3  : -0.749585088887  0.661907995508  0.0  
PC2_1 PC2_2 PC2_3  : -0.661907995508  -0.749585088887  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : -0.00141111109406  0.00141111109406  1.0  
NAXIS : 74  12  12




In [11]:
## Way 1 - 'all'
##---------------
hdr = fits.open(file_sip)[0].header
header = hdr.copy()

print('All kw with the character "3":')
print('********************')
for kw in hdr.keys():
    if '3' in kw:
        del header[kw]
        print(kw)
header['NAXIS'] = 2

print('\n2D WCS (del all kw with "3"):')
print('********************')
print(WCS(header))

All kw with the character "3":
********************
NAXIS3
PC3_3
CRPIX3
CRVAL3
CTYPE3
CUNIT3
PS3_0
PS3_1
A_0_3
A_3_0
B_0_3
B_3_0
AP_0_3
AP_3_0
BP_0_3
BP_3_0
FLXCON03
FLXERR03
GAIN3
BASECH3

2D WCS (del all kw with "3"):
********************
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 148.949761128  69.6707806935  
CRPIX : 41.0  15.5  
PC1_1 PC1_2  : 0.57622073813  0.81729410921  
PC2_1 PC2_2  : -0.81729410921  0.57622073813  
CDELT : -0.000513888895512  0.000513888895512  
NAXIS : 81  30


In [12]:
## Way 2 - default
##-----------------
hdr = fits.open(file_sip)[0].header
header = hdr.copy()

del header['CTYPE3'] # Memory allocation error
del header['NAXIS3']

print('\n2D WCS (del minimum kw):')
print('********************')
print(WCS(header,naxis=2))


2D WCS (del minimum kw):
********************
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 148.949761128  69.6707806935  
CRPIX : 41.0  15.5  
PC1_1 PC1_2  : 0.57622073813  0.81729410921  
PC2_1 PC2_2  : -0.81729410921  0.57622073813  
CDELT : -0.000513888895512  0.000513888895512  
NAXIS : 81  30


In [13]:
## Way 3 - 'sip'
##---------------
hdr = fits.open(file_sip)[0].header
header = hdr.copy()

# header['NAXIS'] = 2 # WCSLIB has detected 3D kw
# del header['NAXIS3']
# del header['PC3_3'] # WCSLIB has detected 3D kw
# del header['CRPIX3'] # WCSLIB has detected 3D kw
# del header['CRVAL3'] # WCSLIB has detected 3D kw
del header['CTYPE3'] # Memory allocation error
# del header['CUNIT3'] # WCSLIB has detected 3D kw
# del header['PS3_0'] # WCSLIB has detected 3D kw
# del header['PS3_1'] # WCSLIB has detected 3D kw
# del header['A_0_3'] # SIP kw
# del header['A_3_0'] # SIP kw
# del header['B_0_3'] # SIP kw
# del header['B_3_0'] # SIP kw
# del header['AP_0_3'] # SIP kw
# del header['AP_3_0'] # SIP kw
# del header['BP_0_3'] # SIP kw
# del header['BP_3_0'] # SIP kw
# del header['FLXCON03']
# del header['FLXERR03']
# del header['GAIN3']
# del header['BASECH3']

for kw in header.keys():
    if ('A_' in kw) and (not 'PA' in kw) and (not 'RA' in kw):
        del header[kw]
    if ('B_' in kw) or ('AP_' in kw) or ('BP_' in kw):
        del header[kw]

print('\n2D WCS (del SIP kw):')
print('********************')
print(WCS(header))
#print(list(header.keys()))


2D WCS (del SIP kw):
********************
WCS Keywords

Number of WCS axes: 3
CTYPE : 'RA---TAN'  'DEC--TAN'  ''  
CRVAL : 148.949761128  69.6707806935  1.0  
CRPIX : 41.0  15.5  1.0  
PC1_1 PC1_2 PC1_3  : 0.57622073813  0.81729410921  0.0  
PC2_1 PC2_2 PC2_3  : -0.81729410921  0.57622073813  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : -0.000513888895512  0.000513888895512  1.0  
NAXIS : 81  30  22


In [14]:
## Testing patch_fits_3D
print('\ndel_kw="all" WCS:')
print('********************')
print(patch_wcs_3D(file_sip, ext='', del_kw='all').wcs)


del_kw="all" WCS:
********************
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 148.949761128  69.6707806935  
CRPIX : 41.0  15.5  
PC1_1 PC1_2  : 0.57622073813  0.81729410921  
PC2_1 PC2_2  : -0.81729410921  0.57622073813  
CDELT : -0.000513888895512  0.000513888895512  
NAXIS : 81  30


In [15]:
print('\ndel_kw=None WCS:')
print('********************')
print(patch_wcs_3D(file_sip, ext='').wcs)


del_kw=None WCS:
********************
WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 148.949761128  69.6707806935  
CRPIX : 41.0  15.5  
PC1_1 PC1_2  : 0.57622073813  0.81729410921  
PC2_1 PC2_2  : -0.81729410921  0.57622073813  
CDELT : -0.000513888895512  0.000513888895512  
NAXIS : 81  30


In [16]:
print('\ndel_kw="sip" WCS:')
print('********************')
print(patch_wcs_3D(file_sip, ext='', del_kw='sip').wcs)


del_kw="sip" WCS:
********************
WCS Keywords

Number of WCS axes: 3
CTYPE : 'RA---TAN'  'DEC--TAN'  ''  
CRVAL : 148.949761128  69.6707806935  1.0  
CRPIX : 41.0  15.5  1.0  
PC1_1 PC1_2 PC1_3  : 0.57622073813  0.81729410921  0.0  
PC2_1 PC2_2 PC2_3  : -0.81729410921  0.57622073813  0.0  
PC3_1 PC3_2 PC3_3  : 0.0  0.0  1.0  
CDELT : -0.000513888895512  0.000513888895512  1.0  
NAXIS : 81  30  22


In [17]:
print('\nNo input error:')
print('********************')
#print(patch_wcs_3D().wcs)


No input error:
********************


## write_hdf5

In [18]:
label = [['alpha', 'beta', 'gamma'], [1,2,3]]
data1 = np.arange(5, 23, 2).reshape((3,3))
# data2 = [1,2,3]
# data2 = [[1,2,3]]
data2 = np.arange(4).reshape((2,2))
write_hdf5(filout, 'Label', label)
write_hdf5(filout, 'Data', data1, append=True)
# write_hdf5(filout, 'Data', data2, ind1=[2,5], append=True)
# write_hdf5(filout, 'Data', data2, ind1=1, ind2=[0,3], append=True)
write_hdf5(filout, 'Data', data2, ind1=[0,2], ind2=[1,3], append=True)
write_hdf5(filout+'_h5grp', 'Grp data', data1, group='/test_grp', verbose=True)
write_hdf5(filout+'_h5grp', 'Grp data', data2, group='/test_grp', 
           ind1=[0,2], ind2=[1,3], append=True, verbose=True)

[write_hdf5] Dataset /test_grp/Grp data has been written in the new file:
     /Users/dhu/Github/RAPYUTA/man/../man/out/test_inout/test_inout_h5grp.h5 .
[write_hdf5] Dataset /test_grp/Grp data in the file:
     /Users/dhu/Github/RAPYUTA/man/../man/out/test_inout/test_inout_h5grp.h5  has been modified.


## read_hdf5

In [19]:
get_label = read_hdf5(filout, 'Label')
print('Label \n', get_label)
get_data = read_hdf5(filout, 'Data')
print('Data \n', get_data)
get_grp = read_hdf5(filout+'_h5grp', 'Grp data', group='test_grp')
print('Grp data \n', get_grp)

Label 
 [['alpha' 'beta' 'gamma']
 ['1' '2' '3']]
Data 
 [[ 5  0  1]
 [11  2  3]
 [17 19 21]]
Grp data 
 [[ 5  0  1]
 [11  2  3]
 [17 19 21]]


## write_ascii

In [20]:
lab = ['col1', 'col2']
arr = np.arange(12).reshape((6,2))
arr_trans = np.arange(0, 12000, 1000).reshape((2,6))
write_ascii(filout, comment='Empty file!')
write_ascii(filout, lab, arr, comment='Data begins from line 3')
write_ascii(filout, dset=arr_trans, trans=True, append=True)
write_ascii(filout+'_mycsv', lab, arr, ext='.csv') # not compatible

## read_ascii

In [21]:
get_arr = read_ascii(filout, dtype=float)
print('col1 \n', get_arr['col1'])
print('col2 \n', get_arr['col2'])
get_adata = read_ascii(datdir+'/filt_IRAC1', start_header=2)
print('col1 - Wave (microns) \n', get_adata['Wave'][:10])
print('col2 - Spectral Response (electrons/photon) \n', get_adata['col2'][:10])

col1 
 [0.e+00 2.e+00 4.e+00 6.e+00 8.e+00 1.e+01 0.e+00 1.e+03 2.e+03 3.e+03
 4.e+03 5.e+03]
col2 
 [1.0e+00 3.0e+00 5.0e+00 7.0e+00 9.0e+00 1.1e+01 6.0e+03 7.0e+03 8.0e+03
 9.0e+03 1.0e+04 1.1e+04]
col1 - Wave (microns) 
 ['3.081060' '3.082890' '3.084730' '3.086560' '3.088400' '3.090240'
 '3.092080' '3.093930' '3.095780' '3.097620']
col2 - Spectral Response (electrons/photon) 
 ['0.000490' '0.000586' '0.000453' '0.000458' '0.000439' '0.000531'
 '0.000641' '0.000442' '0.000535' '0.000655']


## write_csv

In [22]:
write_csv(filout, lab, arr)

## read_csv

In [23]:
get_cdata = read_csv(filout, csvext, 'col2', 'col1')
print('col2: \n', get_cdata['col2'])
print('col1: \n', get_cdata['col1'])

col2: 
 ['1' '3' '5' '7' '9' '11']
col1: 
 ['0' '2' '4' '6' '8' '10']


## fclean

In [24]:
if input('Clean test outputs (y/n): ')=='y':
    info = 'Test outputs is empty! [Done]'
    fclean(outdir, info)

Clean test outputs (y/n): 
